# 5일차: 선형대수 핵심 II - 유사역행렬, SVD, 고유값

## 학습 목표
- 유사역행렬(pseudoinverse) 이해 및 활용
- SVD(특잇값 분해) 개념과 응용
- 고유값/고유벡터 계산
- QR 분해와 행렬 놈(Norm)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
print(f'NumPy 버전: {np.__version__}')

NumPy 버전: 2.2.6


## 💡 이론 개요

### 유사역행렬 (Moore-Penrose Pseudoinverse)
정방행렬이 아니거나 특이행렬일 때도 사용 가능한 일반화된 역행렬

**용도**: 최소제곱 해 구하기 (x = A⁺b)

### SVD (Singular Value Decomposition)
A = UΣVᵀ로 분해
- U: 왼쪽 특이벡터 (m×m 직교행렬)
- Σ: 특잇값 대각행렬 (m×n)
- Vᵀ: 오른쪽 특이벡터 (n×n 직교행렬)

**용도**: 차원 축소, PCA, 압축

### 고유값/고유벡터
Av = λv를 만족하는 λ(고유값)와 v(고유벡터)

**의미**: 선형변환에서 방향이 변하지 않는 벡터와 그 스케일

## 📚 1. 유사역행렬 (Pseudoinverse)

### 1.1 직사각형 행렬의 유사역행렬

In [2]:
# 2×3 행렬 (정방행렬 아님)
A = np.array([[1, 2, 3],
              [4, 5, 6]])

print('행렬 A (2×3):')
print(A)
print(f'Shape: {A.shape}')
print()

# 유사역행렬 계산
A_pinv = np.linalg.pinv(A)
print('유사역행렬 A⁺ (3×2):')
print(A_pinv)
print(f'Shape: {A_pinv.shape}')
print(A@A_pinv)
print(A_pinv@A)

행렬 A (2×3):
[[1 2 3]
 [4 5 6]]
Shape: (2, 3)

유사역행렬 A⁺ (3×2):
[[-0.94444444  0.44444444]
 [-0.11111111  0.11111111]
 [ 0.72222222 -0.22222222]]
Shape: (3, 2)
[[ 1.00000000e+00  3.05311332e-16]
 [-4.44089210e-16  1.00000000e+00]]
[[ 0.83333333  0.33333333 -0.16666667]
 [ 0.33333333  0.33333333  0.33333333]
 [-0.16666667  0.33333333  0.83333333]]


In [3]:
# 유사역행렬 성질 확인
# A⁺A는 입력 공간으로의 투영
AA_plus = A_pinv @ A
print('A⁺A (3×3):')
print(AA_plus)
print()

# AA⁺는 출력 공간으로의 투영  
A_plusA = A @ A_pinv
print('AA⁺ (2×2):')
print(A_plusA)
print()

print('투영 행렬의 성질: M² = M (idempotent)')
print(f'(A⁺A)² == A⁺A: {np.allclose((AA_plus @ AA_plus), AA_plus)}')
print(f'(AA⁺)² == AA⁺: {np.allclose((A_plusA @ A_plusA), A_plusA)}')

A⁺A (3×3):
[[ 0.83333333  0.33333333 -0.16666667]
 [ 0.33333333  0.33333333  0.33333333]
 [-0.16666667  0.33333333  0.83333333]]

AA⁺ (2×2):
[[ 1.00000000e+00  3.05311332e-16]
 [-4.44089210e-16  1.00000000e+00]]

투영 행렬의 성질: M² = M (idempotent)
(A⁺A)² == A⁺A: True
(AA⁺)² == AA⁺: True


### 1.2 최소제곱 문제

In [4]:
# 과대결정 시스템 (방정식 > 미지수)
A = np.array([[1, 1],
              [1, 2],
              [1, 3],
              [1, 4]])
b = np.array([2, 2.5, 3.5, 4])

print('과대결정 시스템:')
print('A (4×2):') ;print(A)
print(f'b: {b}')
print()
A_pinv=np.linalg.pinv(A)
# 유사역행렬로 최소제곱해
x = A_pinv @ b
print('최소제곱해 x:')
print(x)
print()

# 검증
Ax = A @ x
print('Ax (예측):')
print(Ax)
print('\nb (실제):')
print(b)
print('\n잔차 (residual):')
print(b - Ax)
print(f'잔차 제곱합: {np.sum((b - Ax)**2):.6f}')

과대결정 시스템:
A (4×2):
[[1 1]
 [1 2]
 [1 3]
 [1 4]]
b: [2.  2.5 3.5 4. ]

최소제곱해 x:
[1.25 0.7 ]

Ax (예측):
[1.95 2.65 3.35 4.05]

b (실제):
[2.  2.5 3.5 4. ]

잔차 (residual):
[ 0.05 -0.15  0.15 -0.05]
잔차 제곱합: 0.050000


## 📚 2. SVD (특잇값 분해)

### 2.1 기본 SVD

In [5]:
# 3×2 행렬
A = np.array([[3, 1],
              [1, 3],
              [1, 1]], dtype=float)

print('행렬 A (3×2):')
print(A)
print()

# SVD 분해
U, s, Vt = np.linalg.svd(A, full_matrices=True)

print(f'U (3×3): 왼쪽 특이벡터')
print(U)
print(f'Shape: {U.shape}')
print()

print(f's: 특잇값')
print(s)
print(f'Shape: {s.shape}')
print()

print(f'Vᵀ (2×2): 오른쪽 특이벡터')
print(Vt)
print(f'Shape: {Vt.shape}')

행렬 A (3×2):
[[3. 1.]
 [1. 3.]
 [1. 1.]]

U (3×3): 왼쪽 특이벡터
[[-6.66666667e-01  7.07106781e-01 -2.35702260e-01]
 [-6.66666667e-01 -7.07106781e-01 -2.35702260e-01]
 [-3.33333333e-01  5.92577380e-18  9.42809042e-01]]
Shape: (3, 3)

s: 특잇값
[4.24264069 2.        ]
Shape: (2,)

Vᵀ (2×2): 오른쪽 특이벡터
[[-0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]]
Shape: (2, 2)


In [6]:
# SVD 재구성: A = UΣVᵀ
# Σ를 올바른 크기로 만들기
Sigma = np.zeros((3, 2))
Sigma[:2, :2] = np.diag(s)

print('Σ 행렬 (3×2):')
print(Sigma)
print()

# 재구성
A_reconstructed = U @ Sigma @ Vt
print('재구성된 A:')
print(A_reconstructed)
print()

print('원본 A:')
print(A)
print()

print(f'재구성 정확도: {np.allclose(A, A_reconstructed)}')

Σ 행렬 (3×2):
[[4.24264069 0.        ]
 [0.         2.        ]
 [0.         0.        ]]

재구성된 A:
[[3. 1.]
 [1. 3.]
 [1. 1.]]

원본 A:
[[3. 1.]
 [1. 3.]
 [1. 1.]]

재구성 정확도: True


### 2.2 대칭행렬의 SVD

In [7]:
# 대칭행렬
B = np.array([[3, 1],
              [1, 3]], dtype=float)

print('대칭행렬 B:')
print(B)
print()

# SVD
U, s, Vt = np.linalg.svd(B)
print(f'특잇값: {s}')
print()

# 고유값과 비교
eigenvalues = np.linalg.eigvals(B)
print(f'고유값: {np.sort(eigenvalues)[::-1]}')
print()

print('대칭행렬의 특성: 특잇값 = |고유값|')
print(f'일치: {np.allclose(s, np.abs(np.sort(eigenvalues)[::-1]))}')

대칭행렬 B:
[[3. 1.]
 [1. 3.]]

특잇값: [4. 2.]

고유값: [4. 2.]

대칭행렬의 특성: 특잇값 = |고유값|
일치: True


## 📚 3. 고유값과 고유벡터

### 3.1 고유값 분해

In [8]:
# 2×2 행렬의 고유값/고유벡터
A = np.array([[4, 1],
              [2, 3]])

print('행렬 A:')
print(A)
print()

# 고유값과 고유벡터
eigenvalues, eigenvectors = np.linalg.eig(A)

print('고유값 λ:')
print(eigenvalues)
print()

print('고유벡터 (열벡터들):')
print(eigenvectors)
print()

행렬 A:
[[4 1]
 [2 3]]

고유값 λ:
[5. 2.]

고유벡터 (열벡터들):
[[ 0.70710678 -0.4472136 ]
 [ 0.70710678  0.89442719]]



In [9]:
# 검증: Av = λv
for i in range(len(eigenvalues)):
    λ = eigenvalues[i]
    v = eigenvectors[:, i]
    
    Av = A @ v
    λv = λ * v
    
    print(f'고유값 λ{i+1} = {λ:.4f}')
    print(f'고유벡터 v{i+1} = {v}')
    print(f'Av = {Av}')
    print(f'λv = {λv}')
    print(f'일치: {np.allclose(Av, λv)}')
    print()

고유값 λ1 = 5.0000
고유벡터 v1 = [0.70710678 0.70710678]
Av = [3.53553391 3.53553391]
λv = [3.53553391 3.53553391]
일치: True

고유값 λ2 = 2.0000
고유벡터 v2 = [-0.4472136   0.89442719]
Av = [-0.89442719  1.78885438]
λv = [-0.89442719  1.78885438]
일치: True



### 3.2 대칭행렬의 고유분해

In [10]:
# 대칭행렬 (실수 고유값 보장)
S = np.array([[2, 1],
              [1, 2]])

print('대칭행렬 S:')
print(S)
print()

# eigh는 대칭/에르미트 행렬에 최적화
eigenvalues, eigenvectors = np.linalg.eigh(S)

print('고유값 (정렬됨):')
print(eigenvalues)
print()

print('고유벡터 (정규직교):')
print(eigenvectors)
print()

# 직교성 확인
print('고유벡터의 직교성:')
print('v₁·v₂ =', np.dot(eigenvectors[:, 0], eigenvectors[:, 1]))
print('(0에 가까우면 직교)')

대칭행렬 S:
[[2 1]
 [1 2]]

고유값 (정렬됨):
[1. 3.]

고유벡터 (정규직교):
[[-0.70710678  0.70710678]
 [ 0.70710678  0.70710678]]

고유벡터의 직교성:
v₁·v₂ = 0.0
(0에 가까우면 직교)


## 📚 4. PSD 행렬과 Norm

### 4.1 양의 준정부호(PSD) 행렬

In [11]:
# PSD 행렬 생성 (AᵀA는 항상 PSD)
A = np.random.randn(4, 3)
PSD = A.T @ A

print('PSD 행렬 (AᵀA):')
print(PSD)
print()

# 고유값 확인
eigvals = np.linalg.eigvals(PSD)
print('고유값:')
print(eigvals)
print()

print(f'모든 고유값이 비음수: {np.all(eigvals >= -1e-10)}')
print('→ PSD 행렬 확인 ✅')

PSD 행렬 (AᵀA):
[[ 1.67081683 -1.53748474 -1.90510253]
 [-1.53748474  5.77716831  2.49635919]
 [-1.90510253  2.49635919  3.70496533]]

고유값:
[8.28067365 0.52792705 2.34434976]

모든 고유값이 비음수: True
→ PSD 행렬 확인 ✅


### 4.2 행렬 Norm

In [12]:
# 벡터의 norm
v = np.array([3, 4])

print('벡터 v:', v)
print()

# L1 norm (절댓값 합)
L1 = np.linalg.norm(v, ord=1)
print(f'L1 norm: {L1} = |3| + |4| = 7')
print()

# L2 norm (유클리드)
L2 = np.linalg.norm(v, ord=2)
print(f'L2 norm: {L2:.4f} = √(3² + 4²) = 5')
print()

# L∞ norm (최댓값)
Linf = np.linalg.norm(v, ord=np.inf)
print(f'L∞ norm: {Linf} = max(|3|, |4|) = 4')

벡터 v: [3 4]

L1 norm: 7.0 = |3| + |4| = 7

L2 norm: 5.0000 = √(3² + 4²) = 5

L∞ norm: 4.0 = max(|3|, |4|) = 4


In [13]:
# 행렬의 Frobenius norm
M = np.array([[1, 2],
              [3, 4]])

print('행렬 M:')
print(M)
print()

frob = np.linalg.norm(M, 'fro')
print(f'Frobenius norm: {frob:.4f}')
print(f'계산: √(1² + 2² + 3² + 4²) = √30 = {np.sqrt(30):.4f}')

행렬 M:
[[1 2]
 [3 4]]

Frobenius norm: 5.4772
계산: √(1² + 2² + 3² + 4²) = √30 = 5.4772


## 🔥 연습 문제

### 문제 1: SVD로 행렬 재구성

In [14]:
# TODO: 다음 행렬을 SVD 분해하고 재구성하세요
A = np.array([[1, 2],
              [3, 4],
              [5, 6]])

# 여기에 코드 작성
u,s,vt=np.linalg.svd(A)
box=np.zeros((3,2))
box[:2,:2]=np.diag(s)
reoncon_A=u@box@vt
print(A)
print(f"{box}\n")
print(reoncon_A)


[[1 2]
 [3 4]
 [5 6]]
[[9.52551809 0.        ]
 [0.         0.51430058]
 [0.         0.        ]]

[[1. 2.]
 [3. 4.]
 [5. 6.]]


In [15]:
# 해답
A = np.array([[1, 2],
              [3, 4],
              [5, 6]], dtype=float)

print('원본 A:')
print(A)
print()

U, s, Vt = np.linalg.svd(A, full_matrices=False)

print(f'U: {U.shape}')
print(U)
print()

print(f's: {s}')
print()

print(f'Vᵀ: {Vt.shape}')
print(Vt)
print()

# 재구성
Sigma = np.diag(s)
A_recon = U @ Sigma @ Vt

print('재구성:')
print(A_recon)
print(f'\n정확도: {np.allclose(A, A_recon)}')

원본 A:
[[1. 2.]
 [3. 4.]
 [5. 6.]]

U: (3, 2)
[[-0.2298477   0.88346102]
 [-0.52474482  0.24078249]
 [-0.81964194 -0.40189603]]

s: [9.52551809 0.51430058]

Vᵀ: (2, 2)
[[-0.61962948 -0.78489445]
 [-0.78489445  0.61962948]]

재구성:
[[1. 2.]
 [3. 4.]
 [5. 6.]]

정확도: True


### 문제 2: 고유값 계산

In [16]:
# TODO: 다음 행렬의 고유값과 고유벡터를 구하고
# Av = λv를 검증하세요
import numpy as np
B = np.array([[5, 2],
              [2, 5]])

# 여기에 코드 작성
val,vec=np.linalg.eig(B)
print(val)
print(vec)
print(np.allclose(vec@np.diag(val),B@vec))

[7. 3.]
[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]
True


In [17]:
# 해답
B = np.array([[5, 2],
              [2, 5]])

eigvals, eigvecs = np.linalg.eig(B)

print('고유값:', eigvals)
print('\n고유벡터:')
print(eigvecs)
print()

for i in range(2):
    v = eigvecs[:, i]
    λ = eigvals[i]
    
    print(f'\n검증 {i+1}:')
    print(f'Bv = {B @ v}')
    print(f'λv = {λ * v}')
    print(f'일치: {np.allclose(B @ v, λ * v)}')

고유값: [7. 3.]

고유벡터:
[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]


검증 1:
Bv = [4.94974747 4.94974747]
λv = [4.94974747 4.94974747]
일치: True

검증 2:
Bv = [-2.12132034  2.12132034]
λv = [-2.12132034  2.12132034]
일치: True


## 📝 복습

### 핵심 정리

| 개념 | 함수 | 용도 |
|------|------|------|
| 유사역행렬 | `np.linalg.pinv()` | 최소제곱 해 |
| SVD | `np.linalg.svd()` | 차원 축소, PCA |
| 고유분해 | `np.linalg.eig()` | 고유값/벡터 |
| Norm | `np.linalg.norm()` | 크기 측정 |

### 딥러닝 연관
- SVD: PCA, 데이터 압축
- 고유값: Hessian 분석
- Norm: 정규화, 가중치 decay